In [1]:
import sqlalchemy as sq
import geopandas as gpd  # type: ignore
import pandas as pd  # type: ignore
from dotenv import load_dotenv
import os, sys, calendar

sys.path.append("../")
from Shared.DataService import DataService  # type: ignore

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_160096/3953389710.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (callin

In [2]:
TABLENAME = "agg_station_dly"

load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [3]:
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)
conn = db.connect()

In [8]:
# pulling alberta weather sation data from database
query = sq.text("select * FROM public.ab_station_data")
ab_df = pd.read_sql(query, conn)
query = sq.text("select * FROM public.mb_station_data")
mb_df = pd.read_sql(query, conn)
query = sq.text("select * FROM public.sk_station_data")
sk_df = pd.read_sql(query, conn)

In [13]:
df = pd.concat([ab_df, mb_df, sk_df], ignore_index=True)

In [14]:
df

,station_id,date,year,month,day,max_temp,min_temp,mean_temp,total_rain,total_snow,total_precip,snow_on_grnd
0,3020610,2016-06-04,2016,6,4,27.4,8.0,17.7,0.0,0.0,0.0,0.0
1,3020610,2016-06-05,2016,6,5,26.2,7.6,16.9,0.0,0.0,0.0,0.0
2,3020610,2016-06-06,2016,6,6,30.1,8.6,19.4,0.0,0.0,0.0,0.0
3,3020610,2016-06-07,2016,6,7,29.8,14.3,22.1,0.0,0.0,7.8,0.0
4,3020610,2016-06-08,2016,6,8,26.7,11.6,19.2,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3541106,4020286,2008-07-07,2008,7,7,22.4,12.7,17.6,0.0,0.0,0.6,0.0
3541107,4020286,2008-07-08,2008,7,8,22.8,10.8,16.8,0.0,0.0,0.2,0.0
3541108,4020286,2008-07-09,2008,7,9,26.7,8.9,17.8,0.0,0.0,11.8,0.0
3541109,4020286,2008-07-10,2008,7,10,28.5,13.5,21.0,0.0,0.0,9.0,0.0


In [15]:
query = sq.text("select * FROM public.stations_dly")
temp_df = pd.read_sql(query, conn)

In [19]:
temp_df = temp_df[["station_id", "district"]]

In [20]:
temp_df

,station_id,district
0,3010010,4860.0
1,3020035,4810.0
2,3010060,4840.0
3,3010162,4840.0
4,3010237,4840.0
...,...,...
467,4019075,4750.0
468,4019073,4750.0
469,4019085,4750.0
470,4019080,4750.0


In [21]:
df = df.merge(temp_df, on="station_id", how="left")

In [22]:
df

,station_id,date,year,month,day,max_temp,min_temp,mean_temp,total_rain,total_snow,total_precip,snow_on_grnd,district
0,3020610,2016-06-04,2016,6,4,27.4,8.0,17.7,0.0,0.0,0.0,0.0,4820.0
1,3020610,2016-06-05,2016,6,5,26.2,7.6,16.9,0.0,0.0,0.0,0.0,4820.0
2,3020610,2016-06-06,2016,6,6,30.1,8.6,19.4,0.0,0.0,0.0,0.0,4820.0
3,3020610,2016-06-07,2016,6,7,29.8,14.3,22.1,0.0,0.0,7.8,0.0,4820.0
4,3020610,2016-06-08,2016,6,8,26.7,11.6,19.2,0.0,0.0,0.0,0.0,4820.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541106,4020286,2008-07-07,2008,7,7,22.4,12.7,17.6,0.0,0.0,0.6,0.0,4731.0
3541107,4020286,2008-07-08,2008,7,8,22.8,10.8,16.8,0.0,0.0,0.2,0.0,4731.0
3541108,4020286,2008-07-09,2008,7,9,26.7,8.9,17.8,0.0,0.0,11.8,0.0,4731.0
3541109,4020286,2008-07-10,2008,7,10,28.5,13.5,21.0,0.0,0.0,9.0,0.0,4731.0


In [30]:
final_df = df.groupby(["district", "date"]).agg({"max_temp": "mean", "min_temp": "mean", "mean_temp": "mean", "total_rain": ["min", "max", "mean"], "total_snow": ["min", "max", "mean"],"total_precip": ["min", "max", "mean"]}).reset_index()
final_df.columns = [  # type: ignore
    "district",
    "date",
    "max_temp",
    "min_temp",
    "mean_temp",
    "min_rain",
    "max_rain",
    "mean_rain",
    "min_snow",
    "max_snow",
    "mean_snow",
    "min_precip",
    "max_precip",
    "mean_precip"
    
]

In [31]:
final_df

,district,date,max_temp,min_temp,mean_temp,min_rain,max_rain,mean_rain,min_snow,max_snow,mean_snow,min_precip,max_precip,mean_precip
0,4601.0,1992-12-01,2.000000,-6.200000,-2.100000,0.0,0.0,0.0,0.0,0.0,0.000000,1.1,1.1,1.100000
1,4601.0,1992-12-02,-5.900000,-15.700000,-10.800000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,4601.0,1992-12-05,-3.000000,-16.000000,-9.500000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,4601.0,1992-12-06,-7.500000,-20.600000,-14.100000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,4601.0,1992-12-07,-9.500000,-19.400000,-14.500000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921681,4870.0,2022-12-27,-16.286667,-20.503333,-18.380000,0.0,0.0,0.0,0.0,4.0,0.246667,0.0,7.2,2.946667
921682,4870.0,2022-12-28,-13.950000,-19.213333,-16.590000,0.0,0.0,0.0,0.0,0.4,0.013333,0.0,4.0,0.553333
921683,4870.0,2022-12-29,-11.080000,-20.066667,-15.566667,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.8,0.123333
921684,4870.0,2022-12-30,-9.356667,-18.296667,-13.830000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.3,0.010000
